<a href="https://colab.research.google.com/github/bhagesh-codebeast/hackathon/blob/main/OCR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OCR STEPS
### 1. Prepare Image
### 2. Get Model
### 3. Run Model

References:
1. [HF Pipelines](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#computer-vision)
2. [Split scanned document - Try](https://copyprogramming.com/howto/split-text-lines-in-scanned-document)
3. [HF Example OCR](https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/TrOCR/Inference_with_TrOCR_%2B_Gradio_demo.ipynb#scrollTo=4jrTnzzUUPLP)

In [2]:
import warnings,logging
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [3]:
!sudo apt install tesseract-ocr
! pip install pytesseract transformers -q

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (16.4 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

In [4]:
from transformers import pipeline

## Test

## Prepare Image

## Predict Words and sticth them

In [115]:
# ocr = pipeline('image-to-text', model = "microsoft/trocr-large-handwritten")
# ocr2 = pipeline('image-to-text', model = "microsoft/trocr-large-printed")

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

In [6]:
# Working class v0.2
import cv2
import pytesseract
from PIL import Image
import os

class TextImageProcessor:
    def __init__(self, image_path):
        self.image_path = image_path
        self.output_folder = '/content/sample_data/'

    def crop_and_save_line_strips(self):
        image = cv2.imread(self.image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        line_boxes = pytesseract.image_to_data(Image.open(self.image_path), output_type=pytesseract.Output.DICT)
        line_indices = [i for i, text in enumerate(line_boxes['text']) if text.strip()]

        output_folder = os.path.join(self.output_folder, 'line_strips')
        os.makedirs(output_folder, exist_ok=True)

        current_strip = []

        for index in line_indices:
            x, y, w, h = (
                line_boxes['left'][index],
                line_boxes['top'][index],
                line_boxes['width'][index],
                line_boxes['height'][index]
            )

            if not current_strip or y == current_strip[-1]['bottom']:
                current_strip.append({
                    'top': y,
                    'bottom': y + h,
                    'left': x,
                    'right': x + w,
                    'text': line_boxes['text'][index]
                })
            else:
                # Save the current strip and start a new one
                self.save_strip(image_rgb, current_strip, output_folder)
                current_strip = [{
                    'top': y,
                    'bottom': y + h,
                    'left': x,
                    'right': x + w,
                    'text': line_boxes['text'][index]
                }]

        # Save the last strip
        self.save_strip(image_rgb, current_strip, output_folder)

        print(f"Line strips cropped and saved to {output_folder}")

    def save_strip(self, image_rgb, strip, output_folder):
        if strip:
            min_x = min(entry['left'] for entry in strip)
            min_y = min(entry['top'] for entry in strip)
            max_w = max(entry['right'] for entry in strip) - min_x
            max_h = max(entry['bottom'] for entry in strip) - min_y

            # Crop the strip from the original image
            strip_image = image_rgb[min_y:min_y + max_h, min_x:min_x + max_w]

            # Save the strip image
            strip_index = len(os.listdir(output_folder)) + 1
            cv2.imwrite(os.path.join(output_folder, f'strip_{strip_index}.png'), cv2.cvtColor(strip_image, cv2.COLOR_RGB2BGR))

# Example usage:
image_processor = TextImageProcessor('/content/sample_data/test.png')
image_processor.crop_and_save_line_strips()


Line strips cropped and saved to /content/sample_data/line_strips


In [8]:
import os
from PIL import Image
from transformers import TrOCRProcessor
from transformers import VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
path = '/content/sample_data/line_strips'
words = []
sorted_files = sorted(os.listdir(path), key=lambda x: int(x.split('_')[-1].split('.')[0]))
for image in sorted_files:
  print(image)
  image = Image.open(os.path.join(path, image)).convert("RGB")
  pixel_values = processor(image, return_tensors="pt").pixel_values
  generated_ids = model.generate(pixel_values)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  print(generated_text)
  words.append(generated_text)

content = ' '.join(words)
print(content)

strip_1.png
( We,
strip_2.png
the
strip_3.png
People
strip_4.png
cf.
strip_5.png
the
strip_6.png
United
strip_7.png
states.
strip_8.png
inorder.
strip_9.png
to
strip_10.png
form.
strip_11.png
4
strip_12.png
mor e
strip_13.png
erfe's
strip_14.png
union,
strip_15.png
establish
strip_16.png
justice.
strip_17.png
in sure
strip_18.png
do me
strip_19.png
stic.
strip_20.png
Tranqu.
strip_21.png
is
strip_22.png
provide.
strip_23.png
for t
strip_24.png
n e
strip_25.png
common
strip_26.png
ey,
strip_27.png
the
strip_28.png
2017
strip_29.png
rip.
strip_30.png
e vte.
strip_31.png
If are
strip_32.png
and
strip_33.png
sursure
strip_34.png
the
strip_35.png
fess.
strip_36.png
ing
strip_37.png
6f
strip_38.png
o f.
strip_39.png
L.
strip_40.png
00
strip_41.png
beorthy #
strip_42.png
and,
strip_43.png
course (ves
strip_44.png
and
strip_45.png
out in
strip_46.png
of
strip_47.png
eri'
strip_48.png
establish
strip_49.png
this is
strip_50.png
point.
strip_51.png
terity, do
strip_52.png
furt.
strip_53.png
ION


"( We, the People cf. the United states. inorder. to form. 4 mor e erfe's union, establish justice. in sure do me stic. Tranqu. is provide. for t n e common ey, the 2017 rip. e vte. If are and sursure the fess. ing 6f o f. L. 00 beorthy # and, course (ves and out in of eri' establish this is point. terity, do furt. ION for the United. States. of America. etense, to my"